In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

import utils

In [ ]:
#### 2020 ####
column_semantics_2020 = {
    'Investment number': 'investment_id',
    'This investment belongs to project number…': 'project_id',
    'Commissioning Year': 'commissioning_year',
    'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction': 'status',
    'Type of Element': 'asset_type',
    'Substation From': 'substation_1',
    'Substation To': 'substation_2',
    'Technology': 'ac_dc',
    'Total route length (km)': 'specified_length_km',
    'Description of the investment': 'description'
}

status_map_2020 = {
    1: 'under_consideration', # under consideration
    2: 'planned_not_yet_permitting', # planning, not permitting
    3: 'in_permitting', # in permitting
    4: 'under_construction'  # under construction
}
# No need to separate AC and DC as there is a separate column
# in the TYNDP data.
asset_type_map_2020 = {
    'ACTransmissionLine': 'line',
    'DCTransmissionLine': 'line',
    'OnshoreSubstation': 'substation',
    'OffshoreSubstation': 'substation',
    # 'cable' will be mapped to 'line' after adding column 'underground' of binary values.
    'OffshoreDCTransmissionCable': 'cable',
    'OffshoreACTransmissionCable': 'cable'
}

excel_2020, sheet_2020 = '2020/transmission.xlsx', 'Trans.Investments'

In [ ]:
# wanted = tyndp_to_pypsa.prepare_tyndp_data(excel_2010, sheet_2010, column_semantics_2010, status_map_2010, header_row=2)
# wanted = tyndp_to_pypsa.prepare_tyndp_data(excel_2012, sheet_2012, column_semantics_2012, status_map_2012, header_row=4)
#wanted = tyndp_to_pypsa.prepare_tyndp_data(excel_2014, sheet_2014, column_semantics_2014, status_map_2014)
wanted = utils.prepare_tyndp_data(excel_2020, sheet_2020, column_semantics_2020, status_map_2020, asset_type_map=asset_type_map_2020,header_row=1, base_url=r'https://tyndp2020-project-platform.azurewebsites.net/projectsheets/transmission/')
wanted.head()

In [ ]:
# wanted.commissioning_year = utils.commissioning_dates_to_year(wanted.commissioning_year)
# wanted.x1, wanted.y1, wanted.x2, wanted.y2 = np.NAN, np.NAN, np.NAN, np.NAN

In [ ]:
# wanted.to_excel('2020/2020_curated.xlsx')

In [ ]:
# read in newest buses.csv
buses = (pd.read_csv('buses.csv', quotechar="'",
                     true_values='t', false_values='f',
                     dtype=dict(bus_id="str"))
         .set_index("bus_id")
         .drop(['station_id'], axis=1)
         .rename(columns=dict(voltage='v_nom')))
buses = buses.query('symbol == "Substation"')

In [ ]:
# TODO: maybe add option to plot pypsa-eur lines with switch (-> check whether line exists / is placed wrongly)

import plotly.express as px

fig = px.scatter_mapbox(buses, lat="y", lon="x", color_discrete_sequence=["fuchsia"], zoom=3, height=300) # hover_name="City", hover_data=["State", "Population"],
                        # )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
# consecutive coordinates are interpreted as start and endpoints?
lats = [48, 50] 
lons = [10, 12]
fig = px.line_geo(lat=lats, lon=lons)# , color=3*['fuchsia'])
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()